In [1]:
import pandas as pd
import numpy as np
import os 
from PIL import Image
import matplotlib.pyplot as plt

#ignore cameras 10 and 11

def read_spreadsheets():

  cam_list = []
  cam_list_csv = []
  base = 'Cam_'

  print(cam_list)

  for i in range(1, 17):
    cam_list.append(base+str(i))
  # print(cam_list)

  # cam_list.remove('Cam_10')

  spreadsheet_dict = {}

  for cam in cam_list:
    dir = os.path.join('/', 'Users', 'rhyscooper', 'Desktop', 'wildlifeCNN', 'animalCNN', f'{cam}.csv')
    spreadsheet_dict[cam] = pd.read_csv(dir)
  del spreadsheet_dict['Cam_10']
  del spreadsheet_dict['Cam_11']
  return spreadsheet_dict

spreadsheet_dict = read_spreadsheets()


def amend_cam_16_column(spreadsheet_dict):
  c16 = spreadsheet_dict['Cam_16']
  c16 = c16.iloc[:, 1:]

  c16.insert(0, 'Camera no', 'Cam 16')
  spreadsheet_dict['Cam_16'] = c16

amend_cam_16_column(spreadsheet_dict)


def sanity_check():

  sanity_check = [800, 3406, 2540, 426, 4946, 3909, 4076, 3990, 2517, 2262, 2205, 444, 5636, 6624]
  sanity_check_sum = sum(sanity_check)
  print("sanity check", sanity_check_sum,  len(sanity_check))
print(sanity_check())
def build_df(spreadsheet_dict):


  complete_df = pd.concat(spreadsheet_dict.values(), axis=0)
  complete_df.reset_index(drop=True, inplace=True)
  return complete_df

df = build_df(spreadsheet_dict)
# # print(df)
# # print(df.columns)

# def clean_species_values(df):
#   df['Species '] = df['Species '].str.replace(' ', '')
#   df.loc[df['Camera no'] == 'Cam 6', 'Species '] = 'cow'
#   return df

# def get_unique_categorical_values(df, column_name):
#     if column_name in df.columns:
#         unique_values = df[column_name].unique()
#         return unique_values
#     else:
#         return []

# key_mapping = { np.nan:0, 'R':1, 'N':2, 'F':3, 'S':4, 'Owl':5, 'M':6, 'P':7, 'O':8, 'H':9, 'Bi':10, 'Bu':11, 'Jay':12 ,'W':13, 'Dog':14 ,'C':15,
#  'B':16, 'cow': 17}
# def classification_map(df, keymap):
#   df['Species '] = df['Species '].replace(keymap)
#   return df


# df = clean_species_values(df)


# df = classification_map(df, key_mapping)
# # print(df)

# species_list = get_unique_categorical_values(df= df, column_name='Species ')
# print(species_list)

[]
sanity check 43781 14
None


In [2]:
# df.describe()


In [3]:
class Record():
    def __init__(self, dataframe):
        self.df = dataframe
        self.dir =  os.path.join('/', 'Users', 'rhyscooper', 'Desktop', 'wildlifeCNN', 'images',)
        self.num_rows, self.num_columns = self.df.shape
        
    def image_loader(self, index): 
        index_row = np.array(self.df.iloc[index]) 
        # print(index_row)
        filename = index_row[2]
        path = os.path.join (self.dir,filename)
        img = Image.open(path)
        species = index_row[5]
        # if show:
        #     plt.imshow(img)
        #     plt.axis('off')  # Turn off axis labels and ticks
        #     plt.show()
        # if path:
        #     return path, species
        # if not path:
        return img, species 
    
    def train_set_list(self, N_train_samples):
        self.N_train_samples = N_train_samples
        self.train_dir_list = []
        for i in range(0, N_train_samples):
            self.train_dir_list.append((self.image_loader(i)))
            
    def test_set_list(self):
        self.test_dir_list = []
        for i in range(self.N_train_samples, self.num_rows) :
            self.test_dir_list.append((self.image_loader(i)))        
            
    def test_set_list_override(self, N_train_samples_override):
        self.N_train_samples_overide = N_train_samples_override
        self.test_dir_list_override = []
        for i in range(0, N_train_samples_override):
            self.test_dir_list_override.append((self.image_loader(i)))
            

In [4]:
# rcd = Record(dataframe = df)

# rcd.train_set_list(50)
# rcd.test_set_list_override(50)
# # rcd.test_set_list()

# print(len(rcd.train_dir_list))
# print(rcd.train_dir_list)
# # print(len(rcd.test_dir_list))

In [5]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data.dataloader import Dataset
from torch.utils.data import DataLoader, SubsetRandomSampler, ConcatDataset
import albumentations as A
from albumentations.pytorch import ToTensorV2



In [ ]:
class CreateDataset():
    def __init__(self, df, train, N_train_samples):
        self.key_map = { np.nan:0, 'R':1, 'N':2, 'F':3, 'S':4, 'Owl':5, 'M':6, 'P':7, 'O':8, 'H':9, 'Bi':10, 'Bu':11, 'Jay':12 ,'W':13, 'Dog':14 ,'C':15,'B':16, 'cow': 17}
        self.cleaned_df = self.clean_species_values(df)
        self.category_values = self.get_unique_categorical_values(self.cleaned_df)
        self.N_category_values = len(self.category_values)
        self.classification_mapped_df = self.classification_map(self.cleaned_df)
        self.dataset_df = self.classification_mapped_df
        
        
        self.dir =  os.path.join('/', 'Users', 'rhyscooper', 'Desktop', 'wildlifeCNN', 'images',)
        self.num_rows, self.num_columns = self.dataset_df.shape
        
        self.train = train
        self.N_train_samples = N_train_samples
        
        self.dataset = self.dataset_list_compiler(self.train)
    
    def clean_species_values(self, df):
        df['Species '] = df['Species '].str.replace(' ', '')
        # df.loc[df['Camera no'] == 'Cam 6', 'Species '] = 'cow'
        return df

    def get_unique_categorical_values(self, df, column_name = "Species "):
        if column_name in df.columns:
            unique_values = df[column_name].unique()
            return unique_values
        else:
            return []

    def classification_map(self, df):
        df['Species '] = df['Species '].replace(self.key_map)
        return df

    def image_and_label_loader(self, index): 
        index_row = np.array(self.dataset_df.iloc[index]) 
        # print(index_row)
        filename = index_row[2]
        path = os.path.join (self.dir,filename)
        img = Image.open(path)
        species = index_row[5]
        return img, species 
        
    def dataset_list_compiler(self, train):
        image_label_list = []
        if train:
            for i in range(0, self.N_train_samples):
                image_label_list.append((self.image_and_label_loader(i)))        
        if not train:
            image_label_list = []
            for i in range(self.N_train_samples, self.num_rows) :
                image_label_list.append((self.image_and_label_loader(i)))                

        return image_label_list 

In [ ]:
class MyDataset2(torch.utils.data.Dataset):
    '''
    Required class for formatting a dataset and allowing the pytorch DataLoader to retreive images
    in the correct form. Takes as input a dataset of PIL images and a transform to allow the images 
    to be of the right format to be used in subsequent models. Pairs the images with their masks and
    turns them into tensors with shape [3, height, width].
    '''
    def __init__(self, dataset):
        self.dataset = dataset
        self.transform = A.Compose([A.Resize(520, 520, p=1),ToTensorV2()])

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img = self.dataset[idx][0]
        label = self.dataset[idx][1]
        image_np = np.array(img)
        aug = self.transform(image=image_np)
        img=aug['image']

        return img, label
    

In [ ]:
print(df)
train_dataset = CreateDataset(df, True, 30)
train_set = train_dataset.dataset 

output_shape = train_dataset.N_category_values 

      Camera no    No             Filename        Date     Time Species   \
0         Cam 1     1   Cam 1/IMG_0001.JPG         NaN      NaN      NaN   
1         Cam 1     2   Cam 1/IMG_0002.JPG         NaN      NaN      NaN   
2         Cam 1     3   Cam 1/IMG_0003.JPG  12/23/2022  12:49PM      NaN   
3         Cam 1     4   Cam 1/IMG_0004.JPG  12/23/2022  12:49PM      NaN   
4         Cam 1     5   Cam 1/IMG_0005.JPG  12/23/2022  12:49PM      NaN   
...         ...   ...                  ...         ...      ...      ...   
43776    Cam 16  6620  Cam 16/IMG_6620.JPG  01/23/2023  10:15AM      NaN   
43777    Cam 16  6621  Cam 16/IMG_6621.JPG  01/23/2023  10:15AM      NaN   
43778    Cam 16  6622  Cam 16/IMG_6622.JPG  01/23/2023  10:15AM      NaN   
43779    Cam 16  6623  Cam 16/IMG_6623.JPG  01/23/2023  10:15AM      NaN   
43780    Cam 16  6624  Cam 16/IMG_6624.JPG  01/23/2023  10:15AM      NaN   

       Distance 1  Angle 1  Distance 2  Angle 2  Distance 3  Angle 3  
0             Na

In [ ]:
import os
import math 
import numpy as np
import numpy.random as random
import pandas as pd
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torchvision
from torch.utils.data.dataloader import Dataset
from torch.utils.data import DataLoader, SubsetRandomSampler, ConcatDataset
from torch.cuda.amp import autocast, GradScaler
from torchinfo import summary

from sklearn.model_selection import KFold
from PIL import Image
import gc
import torchmetrics

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

TypeError: get_unique_categorical_values() missing 1 required positional argument: 'self'

In [ ]:
class Model():

    def __init__(self, model_name, optim, loss_type, output_shape=18, verbose=False):
        # Initialise class parameters
        self.model_name = model_name
        self.optimiser_type = optim
        self.loss_type = loss_type

        self.model_dic = {
        'DeepLabV3': [torchvision.models.segmentation.DeepLabV3_ResNet101_Weights.DEFAULT,
                        torchvision.models.segmentation.deeplabv3_resnet101],
    
        'FCN' :      [torchvision.models.segmentation.FCN_ResNet101_Weights.DEFAULT,
                        torchvision.models.segmentation.fcn_resnet101],
    
        'LRASPP' :   [torchvision.models.segmentation.LRASPP_MobileNet_V3_Large_Weights.DEFAULT, 
                        torchvision.models.segmentation.lraspp_mobilenet_v3_large],
        }

        # Call the createModel function to return the specified model, and initialise accuracy metrics
        self.createModel(output_shape, verbose)

        
    def createModel(self, output_shape, verbose):
        """Pretrained semantic segmentation model with custom head
        Args:
            output_shape (int, optional): The number of output channels
            in your dataset masks. Defaults to 32.

            verbose (bool, optional): Print out the model architecture. 
            Default is False.

        Returns:
            model: Returns the desired model with either the ResNet101 (for DeepLabV3 and FCN), 
            or MobileNet (for LRASPP) backbone.
        """
        
        self.weights = self.model_dic[self.model_name][0]
        self.model = self.model_dic[self.model_name][1](weights=self.weights)
        self.auto_transform = self.weights.transforms()
        
        # Freeze pretrained "backbone" layers
        if self.model_name == 'LRASPP':
            for name, param in self.model.named_parameters():
                if "backbone" in name:
                    if "14" in name or "15" in name or "16" in name:
                        pass     
                    else:
                        param.requires_grad = False

        if self.model_name == 'DeepLabV3' or self.model_name == 'FCN':
            for name, param in self.model.named_parameters():
                if "backbone" in name:
                    if "layer3" in name or "layer4" in name:
                        pass     
                    else:
                        param.requires_grad = False        


        # Replace the last classifier layer with a Conv2d layer with the correct output shape
        # If the model has an auxiliary classifier, replace the last classifier layer with 32 output channels
        
        if self.model_name == 'DeepLabV3':
            self.model.classifier[-1] = nn.Conv2d(256, output_shape, kernel_size=1, stride=1)
            try:
                self.model.aux_classifier[-1] = nn.Conv2d(256, output_shape, kernel_size=1, stride=1)
                self.model.aux_classifier.add_module('softmax', nn.Softmax(dim=1))
            except:
                pass
        
        if self.model_name == 'FCN':
            self.model.classifier[-1] = nn.Conv2d(512, output_shape, kernel_size=1, stride=1)
            try:
                self.model.aux_classifier[-1] = nn.Conv2d(256, output_shape, kernel_size=1, stride=1)
                self.model.aux_classifier.add_module('softmax', nn.Softmax(dim=1))
            except:
                pass  

        if self.model_name == 'LRASPP':
            self.model.classifier.high_classifier = nn.Conv2d(128, output_shape, kernel_size=1, stride=1)
            self.model.classifier.low_classifier = nn.Conv2d(40, output_shape, kernel_size=1, stride=1)
            try:
                self.model.aux_classifier[-1] = nn.Conv2d(256, output_shape, kernel_size=1, stride=1)
                self.model.aux_classifier.add_module('softmax', nn.Softmax(dim=1))
            except:
                pass

        #Create optimiser and learning rate scheduler
        params = [p for p in self.model.parameters() if p.requires_grad]

        if self.model_name == 'DeepLabV3' or self.model_name == 'FCN':
            if self.optimiser_type == 'SGD':
                self.optimiser = torch.optim.SGD(params, lr=0.0001, momentum=0.9, weight_decay=0.0005)
            if self.optimiser_type == 'Adam': 
                self.optimiser = torch.optim.Adam(params, lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001, amsgrad=False)
            if self.optimiser_type == 'RMSprop':
                self.optimiser = torch.optim.RMSprop(params, lr=0.0001, alpha=0.99, eps=1e-08, weight_decay=0.001, momentum=0.9)
            self.lr_scheduler = torch.optim.lr_scheduler.StepLR(self.optimiser, step_size=4, gamma=0.01) 

        if self.model_name == 'LRASPP':
            if self.optimiser_type == 'SGD':
                self.optimiser = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
            if self.optimiser_type == 'Adam': 
                self.optimiser = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001, amsgrad=False)
            if self.optimiser_type == 'RMSprop':
                self.optimiser = torch.optim.RMSprop(params, lr=0.001, alpha=0.99, eps=1e-08, weight_decay=0.001, momentum=0.9)
            self.lr_scheduler = torch.optim.lr_scheduler.StepLR(self.optimiser, step_size=3, gamma=0.01)

        # Initialise either the weighted Cross Entropy Loss or unweighted
        if self.loss_type == 'Standard_CEL':
            self.loss = nn.CrossEntropyLoss()
        # if self.loss_type == 'Weighted_CEL':
            # self.loss = nn.CrossEntropyLoss(weight=class_weights)
        
        #Optionally print out the new model architecture
        if verbose:
            print(summary(model=self.model, 
                input_size=(10, 3, 520, 520),
                col_names=["input_size", "output_size", "num_params", "trainable"],
                col_width=20,
                row_settings=["var_names"]
            )) 

        return self.model

m1 = Model('DeepLabV3', 'Adam', 'Standard_CEL', verbose=False)
m2 = Model('FCN', 'Adam', 'Standard_CEL', verbose=False)
m3 = Model('LRASPP', 'Adam', 'Standard_CEL', verbose=False)


In [ ]:
#Garbage collect and empty cache
gc.collect()
torch.cuda.empty_cache()

class TrainTest():
  def __init__(self, model_class, trainLoader, testLoader, n_epochs=3, batch_size=3, k=10):
    
    # Initialise model specific attributes
    self.model_class = model_class
    self.model = self.model_class.model.to(device)
    self.lr_scheduler = self.model_class.lr_scheduler
    self.optimiser = self.model_class.optimiser
    self.loss = self.model_class.loss
    # self.iou_accuracy = self.model_class.iou_acc
    # self.pixel_accuracy = self.model_class.pix_acc


    # Initialise remaining attributes
    self.trainLoader = trainLoader
    self.testLoader = testLoader
    self.n_epochs = n_epochs
    self.batch_size = batch_size
    self.k = k

    # Check to see whether the combination of batch size and dataloader size will result in a batch size of 1 during training,
    # which will break the model. Change the batch size if so.
    while len(self.trainLoader)*(self.k - 1)/self.k % self.batch_size == 1:
        proceed = input(f"Invalid batch size. Proceed with batch_size = {self.batch_size + 1}?  [Y/N]\n")
        if proceed.lower() == 'y':
            self.batch_size += 1
            break
        self.batch_size = input("Enter batch size: ")
    
  def train_epoch(self, dataloader):
          self.model.train()
        #   iou_acc = 0.0
        #   pix_acc = 0.0
          train_loss = 0.0

          scaler = GradScaler()

          for idx, (images, labels) in enumerate(dataloader): 
              images, labels = images.to(device).float(), labels.to(device)
              images, labels = images.to(device).float(), labels.to(device)

              # Calculate output predicitons
              self.optimiser.zero_grad()
              with autocast(dtype=torch.float16): # Use automatic mixed precision training to optimise memory usage
                  outputs = self.model(images)['out']
                
                  # Convert labels to 32 channel one-hot encoding
                #   labels = encode_segmap(labels)

                  # Calculate loss 
                  loss = self.loss(outputs, labels.float())
                    
              # Perform scaled backward pass and update loss and accuracy
              scaler.scale(loss).backward()
              scaler.step(self.optimiser)
              scaler.update()
              train_loss += loss.item() * images.size(0)
            #   iou_acc += self.iou_accuracy(outputs, labels).item() * images.size(0)
            #   pix_acc += self.pixel_accuracy(outputs, labels).item() * images.size(0)

          # Update learning rate scheduler
          self.lr_scheduler.step()

        #   return iou_acc, pix_acc, train_loss
          return  train_loss
      
  def validation_epoch(self, dataloader):
      
      self.model.eval()
      validation_iou_acc = 0.0
      validation_pix_acc = 0.0
      validation_loss = 0.0
      
      with torch.no_grad():
          for idx, (images, labels) in enumerate(dataloader): 
              images, labels = images.to(device).float(), labels.to(device)
    
              # Calculate output predicitons
              outputs = self.model(images)['out']

              # Convert labels to 32 channel one-hot encoding
              labels = encode_segmap(labels)

              # Calculate accuracy and loss
              loss = self.loss(outputs, labels.float())
              validation_loss += loss.item() * images.size(0)
              validation_iou_acc += self.iou_accuracy(outputs, labels).item() * images.size(0)
              validation_pix_acc += self.pixel_accuracy(outputs, labels).item() * images.size(0)

      return validation_iou_acc, validation_pix_acc, validation_loss

  def k_fold_train(self, shuffle=True, verbose=True):

      kfold = KFold(n_splits=self.k, shuffle=shuffle)
      cv_average_train_iou_accuracy = []
      cv_average_train_pixel_accuracy = []
      cv_average_validation_iou_accuracy = []
      cv_average_validation_pixel_accuracy = []
      cv_average_train_loss = []
      cv_average_validation_loss = []
      start_time = time.time()

      for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(self.trainLoader.dataset)):
          print("Current Fold:", fold_idx + 1)

          # Create data loaders for the training and validation subsets
          train_loader = DataLoader(self.trainLoader, batch_size=self.batch_size, sampler=SubsetRandomSampler(train_idx))
          val_loader = DataLoader(self.trainLoader, batch_size=self.batch_size, sampler=SubsetRandomSampler(val_idx))
          
          # Initialse lists to keep track of accuracy at each epoch
          epoch_train_iou_accuracy = []
          epoch_train_pixel_accuracy = []
          epoch_validation_iou_accuracy = []
          epoch_validation_pixel_accuracy = []
          epoch_train_loss = []
          epoch_validation_loss = []

          for epoch in range(self.n_epochs):

              train_cumulative_iou_acc, train_cumulative_pix_acc, train_loss = TrainTest.train_epoch(self, train_loader)
              validation_cumulative_iou_acc, validation_cumulative_pix_acc, validation_loss = TrainTest.validation_epoch(self, val_loader)
              
              # Find the IoU and Pixel accuracy, and loss
              train_iou_acc = train_cumulative_iou_acc / len(train_loader.sampler) * 100
              train_pix_acc = train_cumulative_pix_acc / len(train_loader.sampler) * 100
              validation_iou_acc = validation_cumulative_iou_acc / len(val_loader.sampler) * 100
              validation_pix_acc = validation_cumulative_pix_acc / len(val_loader.sampler) * 100
              train_loss = train_loss / len(train_loader.sampler)
              validation_loss = validation_loss / len(val_loader.sampler)

              if verbose==True:
                  print("Epoch:{}/{} Training IoU Acc: {:.2f}%, Validation IoU Acc: {:.2f}%, Training Pixel Acc: {:.2f}%, Validation Pixel Acc: {:.2f}%, Training Loss: {:.3f}, Validation Loss: {:.3f}".format(
                            epoch + 1, self.n_epochs, train_iou_acc, validation_iou_acc, train_pix_acc, validation_pix_acc, train_loss, validation_loss))
              
              # Append epoch lists keeping track of accuracy and loss at each epoch 
              epoch_train_iou_accuracy.append(train_iou_acc)
              epoch_train_pixel_accuracy.append(train_pix_acc)
              epoch_validation_iou_accuracy.append(validation_iou_acc)
              epoch_validation_pixel_accuracy.append(validation_pix_acc)
              epoch_train_loss.append(train_loss)
              epoch_validation_loss.append(validation_loss)
          
          # Append the CV average lists with the list containing the accuracy and loss for the epochs just gone
          cv_average_train_iou_accuracy.append(epoch_train_iou_accuracy)
          cv_average_train_pixel_accuracy.append(epoch_train_pixel_accuracy)
          cv_average_validation_iou_accuracy.append(epoch_validation_iou_accuracy)
          cv_average_validation_pixel_accuracy.append(epoch_validation_pixel_accuracy)
          cv_average_train_loss.append(epoch_train_loss)
          cv_average_validation_loss.append(epoch_validation_loss)

      # Calculate and print the training time
      train_time = time.time() - start_time
      print(f'The time taken to train the network was {int(train_time // 60)} mins {train_time % 60 :.0f} seconds')

      # Find the average accuracy and loss for each epoch (averaging across k folds)
      cv_average_train_iou_accuracy = np.mean(cv_average_train_iou_accuracy, axis=0)
      cv_average_train_pixel_accuracy = np.mean(cv_average_train_pixel_accuracy, axis=0)
      cv_average_validation_iou_accuracy = np.mean(cv_average_validation_iou_accuracy, axis=0)
      cv_average_validation_pixel_accuracy = np.mean(cv_average_validation_pixel_accuracy, axis=0)
      cv_average_train_loss = np.mean(cv_average_train_loss, axis=0)
      cv_average_validation_loss = np.mean(cv_average_validation_loss, axis=0)
      
      self.cv_average_train_iou_accuracy, self.cv_average_train_pixel_accuracy = cv_average_train_iou_accuracy, cv_average_train_pixel_accuracy
      self.cv_average_validation_iou_accuracy, self.cv_average_validation_pixel_accuracy = cv_average_validation_iou_accuracy, cv_average_validation_pixel_accuracy
      self.cv_average_train_loss, self.cv_average_validation_loss = cv_average_train_loss, cv_average_validation_loss

      return cv_average_train_iou_accuracy, cv_average_train_pixel_accuracy, cv_average_validation_iou_accuracy, cv_average_validation_pixel_accuracy, cv_average_train_loss, cv_average_validation_loss

  def test(self):

      self.test_loader = DataLoader(self.testLoader, batch_size=self.batch_size, shuffle=True)
      
      test_iou_acc, test_pix_acc = 0.0, 0.0
      test_loss = 0.0
      self.model.eval()

      with torch.no_grad():
          for idx, (images, labels) in enumerate(self.test_loader):
              if idx == len(self.testLoader) // self.batch_size:   
                  images_copy, labels_copy = images.to("cpu").clone(), labels.to("cpu").clone()
              images, labels = images.to(device).float(), labels.to(device)
              
              # Calculate output predicitons
              outputs = self.model(images)['out']
  
              # Convert labels to 32 channel one-hot encoding
              labels = encode_segmap(labels)
  
              # Calculate accuracy and loss
              test_iou_acc += self.iou_accuracy(outputs, labels).item() * images.size(0)
              test_pix_acc += self.pixel_accuracy(outputs, labels).item() * images.size(0)
              loss = self.loss(outputs, labels.float())
              test_loss += loss.item() * images.size(0)
      
      # Calculate and print the final testing accuracy and loss
      test_iou_acc = test_iou_acc / len(self.testLoader) * 100
      test_pix_acc = test_pix_acc / len(self.testLoader) * 100
      test_loss = test_loss / len(self.testLoader)

      print(f"Final Test IoU Accuracy: {test_iou_acc:.2f}%, Final Test Pixel Accuracy: {test_pix_acc:.2f}%, Final Test Loss: {test_loss:.3f}")
      
      # Select random image from last batch and display predicted mask alongside original mask and image
      idx = random.randint(0, images.shape[0])
      original_image = np.transpose(images_copy[idx], (1, 2, 0))
      true_mask = np.transpose(labels_copy[idx], (1, 2, 0))
      outputs_copy = self.model(images)['out'].cpu()
      outputs_copy = outputs_copy.detach()
      predicted_mask = one_hot_to_rgb(outputs_copy[idx])
      
      plt.figure(figsize=(6,6))
      
      ax = plt.subplot2grid((2,4),(0,0), colspan=2)
      ax.imshow(original_image)
      plt.title("Original Image")
      plt.axis("off")
  
      # Convert a one-hot image to RGB and display
      ax1 = plt.subplot2grid((2,4),(0,2), colspan=2)
      ax1.imshow(true_mask)
      plt.title("True Mask")
      plt.axis("off")
  
      ax2 = plt.subplot2grid((2,4),(1,1), colspan=2)
      ax2.imshow(predicted_mask)
      plt.title("Predicted Mask")
      plt.axis("off")
  
      plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0.2)
      plt.show()
  
  def plot(self):

      # Plot graph showing the IoU and Pixel accuracy for the training and validation sets across n epochs
      x = [str(i+1) for i in range(self.n_epochs)]
      plt.figure()
      plt.title(f"Cross-Validated Average Training Accuracy over {self.n_epochs} epochs")
      plt.plot(x, self.cv_average_train_iou_accuracy, label='Training IoU Accuracy')
      plt.plot(x, self.cv_average_train_pixel_accuracy, label='Training Pixel Accuracy')
      plt.plot(x, self.cv_average_validation_iou_accuracy, label='Validation IoU Accuracy')
      plt.plot(x, self.cv_average_validation_pixel_accuracy, label='Validation Pixel Accuracy')
      plt.xlabel("Epoch")
      plt.ylabel("Accuracy (%)")
      plt.legend()
      plt.show()

      # Plot graph showing the training and validation loss over n epochs
      plt.figure()
      plt.title(f"Cross-Validated Average Training Loss over {self.n_epochs} epochs")
      plt.plot(x, self.cv_average_train_loss, color='red', label='Training Loss')
      plt.plot(x, self.cv_average_validation_loss, color='blue', label='Validation Loss')
      plt.xlabel("Epoch")
      plt.ylabel("Loss")
      plt.legend()        
      plt.show()

# run = TrainTest(deepLabV3Model, trainLoader, testLoader, n_epochs=2, batch_size=8, k=3)
# run.k_fold_train()
# run.test()
# run.plot()


In [ ]:
TT = TrainTest(m1, train_loader, test_loader, n_epochs=2, batch_size=8, k=3)
TT.train_epoch(dataloader=train_loader)

/Users/rhyscooper/opt/anaconda3/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


AttributeError: to